In [2]:
!pip install tensorflow-io

     |████████████████████████████████| 25.3MB 134kB/s 


In [3]:
!pip install pydicom

     |████████████████████████████████| 1.9MB 13.6MB/s 


In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import tensorflow as tf

AUTO = tf.data.experimental.AUTOTUNE
# Detect hardware, return appropriate distribution strategy
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [6]:
import warnings
warnings.filterwarnings("ignore")
import matplotlib.pyplot as plt
import random
from PIL import Image
import seaborn as sns
import numpy as np
import re
import datetime as dt
from datetime import datetime
import pydicom as dicom
from tqdm.notebook import tqdm
from glob import glob
import pandas as pd
import tensorflow_io as tfio
import tensorflow as tf
from keras.models import load_model
import time
import logging, os
from joblib import Parallel, delayed
logging.disable(logging.WARNING)
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "3"
import tensorflow as tf
from tensorflow.keras import backend as K
from tensorflow.keras.losses import binary_crossentropy
import random, re, math
import tensorflow as tf, tensorflow.keras.backend as K
from tensorflow.keras.layers import Dense
from tensorflow.keras.models import Model
from tensorflow.keras import optimizers
from tensorflow.keras.models import Sequential
import tensorflow.keras.layers as L
from tensorflow.keras.applications import ResNet152V2, InceptionResNetV2, InceptionV3, Xception, VGG19
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import Dense, Input, Dropout, Lambda, Conv2D, Conv2DTranspose, MaxPooling2D, Concatenate, Activation, Add, multiply, add, concatenate, LeakyReLU, ZeroPadding2D, UpSampling2D, BatchNormalization, Flatten
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from tensorflow.keras.losses import binary_crossentropy
from tensorflow.keras import backend as K 

from sklearn.metrics import classification_report, confusion_matrix
# imports for building the network
from tensorflow import reduce_sum
from tensorflow.keras.backend import pow
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv2D, MaxPool2D, UpSampling2D, Concatenate, Add, Flatten
from sklearn.model_selection import train_test_split
import cv2
import matplotlib.pyplot as plt
from tensorflow.keras.applications import DenseNet121, ResNet50V2, DenseNet169, InceptionResNetV2,MobileNetV2, NASNetMobile, DenseNet201, NASNetLarge, Xception
from tensorflow.keras.layers import Average

In [9]:
from glob import glob
import glob

# reading all dcm files into train and text
train = sorted(glob.glob("/content/drive/MyDrive/siim-acr-pneumothorax/pneumothorax/dicom-images-train/*/*/*.dcm"))
test = sorted(glob.glob("/content/drive/MyDrive/siim-acr-pneumothorax/pneumothorax/dicom-images-test/*/*/*.dcm"))
# reading the csv
dataset = pd.read_csv('/content/drive/MyDrive/siim-acr-pneumothorax/pneumothorax/train-rle.csv', delimiter=",")

In [10]:
import pydicom

#dataframe to ease the access
patients = []
remove=[]
pd.reset_option('max_colwidth')

for t in train:
  data = pydicom.dcmread(t)
  patient = {}
  patient["UID"] = data.SOPInstanceUID
  
  try:
    encoded_pixels = dataset[dataset["ImageId"] == patient["UID"]].values[0][1]
    patient["EncodedPixels"] = encoded_pixels
  except:
    remove.append("/content/drive/MyDrive/siim-acr-pneumothorax/pneumothorax/dicom-images-train/" + data.StudyInstanceUID + "/" + data.SeriesInstanceUID + "/" + data.SOPInstanceUID + ".dcm")
  patient["Age"] = data.PatientAge
  patient["Sex"] = data.PatientSex
  patient["Modality"] = data.Modality
  patient["BodyPart"] = data.BodyPartExamined
  patient["ViewPosition"] = data.ViewPosition
  patient["path"] = "/content/drive/MyDrive/siim-acr-pneumothorax/pneumothorax/dicom-images-train/" + data.StudyInstanceUID + "/" + data.SeriesInstanceUID + "/" + data.SOPInstanceUID + ".dcm"
  patients.append(patient)

patients_train = pd.DataFrame(patients,columns=["EncodedPixels","path"])
patients_train=patients_train.loc[~patients_train['path'].isin(remove)] #remove rows which do not have images

In [48]:
train_path=[]
train_encoded_pixel=[]
test_path=[]
test_encoded_pixel=[]
train_len=len(patients_train)-int(len(patients_train)*0.2) #We are dividing the data into train and test
test_len=int(len(patients_train)*0.2)
count=0
for i in tqdm(range(0,len(patients_train))):
  if count<=train_len:
    train_path.append(list(patients_train['path'].values)[i])
    train_encoded_pixel.append(list(patients_train['EncodedPixels'].values)[i])
    count=count+1
  else:
    test_path.append(list(patients_train['path'].values)[i])
    test_encoded_pixel.append(list(patients_train['EncodedPixels'].values)[i])

# **Function 1**

In [49]:
#Function-1
def final_fun_1(X):
  img = tf.io.read_file(X)
  image = tfio.image.decode_dicom_image(img, dtype=tf.uint8,color_dim=True,scale='preserve')
  image = tf.image.convert_image_dtype(image, tf.float32)#converting the image to tf.float32
  image=tf.squeeze(image,[0]) #squeezing the image because the file is of the shape(1,1024,1024,1) and we want (1024,1024,3)
  b = tf.constant([1,1,3], tf.int32)
  image=tf.tile(image,b)#the image is of the shape (1024,1024,1) to make it (1024,1024,3) I am using tf.tile
  image=tf.image.resize(image,size=[256,256])
  image=tf.expand_dims(image,axis=0)
   #recall
  if model.predict(image)>=0.5:
    print("Pneumothorax has been detected")
    mask=final.predict(image)
    mask=(mask>0.5).astype(np.uint8)
    plt.figure(figsize=(20,6))
    plt.title("Mask")
    return plt.imshow(np.squeeze(mask),cmap='gray')
  else:
    return "No Pneumothorax Detected"

# **Function 2**

In [50]:
def final_fun_2(X,Y):
  img = tf.io.read_file(X)
  image = tfio.image.decode_dicom_image(img, dtype=tf.uint8,color_dim=True,scale='preserve')
  image = tf.image.convert_image_dtype(image, tf.float32)#converting the image to tf.float32
  image=tf.squeeze(image,[0]) #squeezing the image because the file is of the shape(1,1024,1024,1)and we want (1024,1024,3)
  b = tf.constant([1,1,3], tf.int32)
  image=tf.tile(image,b)#the image is of the shape (1024,1024,1) to make it (1024,1024,3) and for this using tf.tile
  image=tf.image.resize(image,size=[256,256])
  image=tf.expand_dims(image,axis=0)
  if Y!=" -1":
    print("Ground truth of Classification is 1(Has Pneumothorax)")
    print('*'*100)
  else:
    print("Ground truth of Classification is 0(Does not have Pneumothorax)")
    print("Ground truth of Segmentation -- There is no mask")
    print('*'*100)

    
  if model.predict(image)>=0.5:
    print("Pneumothorax has been detected")
    mask=final.predict(image)
    mask=(mask>0.5).astype(np.uint8)
    try:
      true_mask=Image.fromarray(mask_functions.rle2mask(Y,1024,1024).T).resize((256,256), resample=Image.BILINEAR)
      true_mask=np.array(true_mask)
      plt.figure(figsize=(20,6))
      plt.subplot(121)
      plt.title("X-ray image with mask(Ground truth)")
      plt.imshow(np.squeeze(image),cmap='gray')
      plt.imshow(np.squeeze(true_mask),cmap='gray',alpha=0.3)
      plt.subplot(122)
      plt.title("X-ray image with mask(Predicted)")
      plt.imshow(np.squeeze(image),cmap='gray')
      plt.imshow(np.squeeze(mask),cmap='gray',alpha=0.3)
      return plt.show()
    except: #if there is no ground truth mask
      plt.figure(figsize=(20,6))
      plt.title("X-ray image with mask(Predicted)")
      plt.imshow(np.squeeze(image),cmap='gray')
      plt.imshow(np.squeeze(mask),cmap='gray',alpha=0.3)
      return plt.show()

In [51]:
print(len(test_path))

2136


# **Define the Metric**

In [52]:
import keras.backend as K
def dice_coef(y_true, y_pred, smooth=1):
  y_true_f = K.flatten(y_true)
  y_pred_f = K.flatten(y_pred)
  intersection = K.sum(y_true_f * y_pred_f)
  return (2. * intersection + smooth) / (K.sum(y_true_f) + K.sum(y_pred_f) + smooth)

In [53]:
import random
import os
#Selecting a random value from the test data
rand_id = random.randrange(0,len(test_path))
# rand_id = np.random.randint(len(test_path))
# print(rand_id)
model=load_model('/content/drive/MyDrive/siim-acr-pneumothorax/pneumothorax/densenet121_weights-33-0.9072.hdf5')
final=tf.keras.models.load_model('/content/drive/MyDrive/siim-acr-pneumothorax/pneumothorax/unet_with_densenet_weights-23-0.4608.hdf5',custom_objects={'dice_coef':dice_coef})

In [54]:
start_time = time.time()
final_fun_1(test_path[rand_id])
print("--- %s seconds ---" % (time.time() - start_time))

--- 1.4749116897583008 seconds ---


In [55]:
start_time = time.time()
final_fun_2(test_path[rand_id],test_encoded_pixel[rand_id])
print("--- %s seconds ---" % (time.time() - start_time))

Ground truth of Classification is 0(Does not have Pneumothorax)
Ground truth of Segmentation -- There is no mask
****************************************************************************************************
--- 0.0878300666809082 seconds ---
